In [11]:
import requests
import pandas as pd
import json
import csv
from geojson import Feature, FeatureCollection, Point

In [5]:
def converter(csv_filepath, pred_filepath, output_path, date_col, y_col):

    data_df = pd.read_csv(csv_filepath)
    pred_df = pd.read_csv(pred_filepath)
    
    features = []

    for row in data_df.iterrows():
        features.append(
            {
                "date": "new Date("+row[1][date_col]+")",
                "observed": row[1][y_col]
            }
        )

    for row in pred_df.iterrows():    
        features.append(
            {
                "date": "new Date("+row[1][date_col]+")",
                "easing": row[1]["easing"],
                "maintaining": row[1]["maintaining"],
                "increasing": row[1]["increase"] 
            }
        )
    with open(output_path, "w") as f:
        f.write("%s" % features)

In [12]:
snow_pred_df = pd.read_csv("static/data/snow_pred_df.csv")
snow_pred_df = snow_pred_df.rename(columns={"easing": "SnowEasing", "maintaining": "SnowMaintaining", "increase": "SnowIncrease"})
snow_pred_df

,Year,SnowEasing,SnowMaintaining,SnowIncrease
0,2011,708.187450,699.622264,691.057078
1,2012,708.238535,691.103928,673.960753
2,2013,708.293849,682.585593,656.851616
3,2014,708.353388,674.067257,639.729660
4,2015,708.417147,665.548922,622.594874
...,...,...,...,...
186,2197,786.129041,-884.788140,-2718.555155
187,2198,786.897063,-893.306475,-2738.187111
188,2199,787.668577,-901.824811,-2757.833561
189,2200,788.443580,-910.343146,-2777.494516


In [13]:
prcp_pred_df = pd.read_csv("static/data/PRCP_pred_df.csv")
prcp_pred_df = prcp_pred_df.rename(columns={"easing": "prcpEasing", "maintaining": "prcpMaintaining", "increase": "prcpIncrease"})
prcp_pred_df

,Year,prcpEasing,prcpMaintaining,prcpIncrease
0,2011,848.087568,847.642706,847.197843
1,2012,848.090221,847.200277,846.309887
2,2013,848.093094,846.757848,845.421265
3,2014,848.096187,846.315418,844.531977
4,2015,848.099498,845.872989,843.642023
...,...,...,...,...
186,2197,852.135733,765.350897,670.107894
187,2198,852.175623,764.908468,669.088240
188,2199,852.215694,764.466039,668.067834
189,2200,852.255947,764.023609,667.046675


In [16]:
pred_df = snow_pred_df.merge(prcp_pred_df, how="left", on="Year")
pred_df

,Year,SnowEasing,SnowMaintaining,SnowIncrease,prcpEasing,prcpMaintaining,prcpIncrease
0,2011,708.187450,699.622264,691.057078,848.087568,847.642706,847.197843
1,2012,708.238535,691.103928,673.960753,848.090221,847.200277,846.309887
2,2013,708.293849,682.585593,656.851616,848.093094,846.757848,845.421265
3,2014,708.353388,674.067257,639.729660,848.096187,846.315418,844.531977
4,2015,708.417147,665.548922,622.594874,848.099498,845.872989,843.642023
...,...,...,...,...,...,...,...
186,2197,786.129041,-884.788140,-2718.555155,852.135733,765.350897,670.107894
187,2198,786.897063,-893.306475,-2738.187111,852.175623,764.908468,669.088240
188,2199,787.668577,-901.824811,-2757.833561,852.215694,764.466039,668.067834
189,2200,788.443580,-910.343146,-2777.494516,852.255947,764.023609,667.046675


In [24]:
# combine precip & snow
data_df = pd.read_csv("static/data/dfFilt.csv")
# pred_df = pd.read_csv(pred_filepath)

features = []

for row in data_df.iterrows():
    features.append(
        {
            "year": int(row[1]["Year"]),
            "precipitation": row[1]["PRCP"],
            "snow fall": row[1]["SNOW"]
        }
    )

for row in pred_df.iterrows():    
    features.append(
        {
            "date": int(row[1]["Year"]),
            "precipitation_e": row[1]["prcpEasing"],
            "precipitation_u": row[1]["prcpMaintaining"],
            "precipitation_i": row[1]["prcpIncrease"],
            "snow_e": row[1]["SnowEasing"],
            "snow_u": row[1]["SnowMaintaining"],
            "snow_i": row[1]["SnowIncrease"],
        }
    )
with open("static/data/snow_prcp.json", "w") as f:
    f.write("%s" % features)